In [1]:
import pandas as pd
df = pd.read_csv("/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/input data/retweet_data_combined_labeled_by_retweet.csv")
df.head()

,timestamp,tweet,retweet_author,original_author,retweet_lc,original_lc,retweet_party
0,2020-06-18 11:59:57+00:00,PM ⁦@narendramodi⁩ to launch Garib Kalyan Rojg...,BJP4AnN,PMOIndia,bjp4ann,pmoindia,BJP
1,2020-05-20 18:15:42+00:00,A step towards economic resilience #aatmanirbh...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP
2,2020-05-20 18:14:57+00:00,.@MOFPI_GOI wholeheartedly welcomes the Cabine...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP
3,2020-06-02 15:58:00+00:00,राज्यपाल भगतसिंह कोश्यारी यांनी मुख्यमंत्री उद...,Nitinkalebjp,maha_governor,nitinkalebjp,maha_governor,BJP
4,2020-12-15 09:47:14+00:00,VIDEO : Pune | केंद्रीय मंत्री प्रकाश जावडेकरा...,VartakKuldeep,TV9Marathi,vartakkuldeep,tv9marathi,INC


In [2]:
import pandas as pd
from pathlib import Path

CSV_PATH = "/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/input data/retweet_data_combined_labeled_by_retweet.csv"

INFLUENCER_COL = "original_author"   # the account whose tweet got retweeted
PARTY_COL      = "retweet_party"     # party of the retweeting politician
TIME_COL       = "timestamp"

# ──────────────────────────────────────────────────────────────────────────────
# 1) Load & prep
df = pd.read_csv(CSV_PATH)
df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce", utc=True)
df["year"] = df[TIME_COL].dt.year

# 2) Party blocs (uppercase normalization). You said ruling includes RSS & ABVP.
pro_ruling = {
    "BJP","JDU","LJP","HAM","JSP","NPP","AIADMK","ADMK","AJSU","AGP","RPI","NPF",
    "IPFT","AD (Soneylal)","NDPP","RSS","NDA","MNF","VHP","YSRCP","ABVP","BJD",
    "BSCP","JVM(P)"
}

pro_opp = {
    "INC","RSP","AITC","TMC","DMK","SP","CPIM","RJD","GFP","RLP","JMM","JKNC",
    "IUML","CPI","VCK","CPIML","MNM","AAP","AIMIM","AIUDF","ASP","SAMAJWADI",
    "SDF","TJS","TRS","Shiv Sena","SBSP","TDP","JDS","DMDK","NCP","RLD","SKM",
    "AMMK","IJK","AISMK","JANATA DALAL","DFYI","JJP","YKD","AIPF","BVA","JAPL",
    "JCC","LSP","SPI","SHP","SWI","LDP","SA","JSVP","KSP","SFI","JKYNC"
}

def side_from_party(p):
    p = str(p).upper().strip()
    if p in pro_ruling: return "ruling"
    if p in pro_opp:    return "opposition"
    return "other"

df["side"] = df[PARTY_COL].apply(side_from_party)

# ──────────────────────────────────────────────────────────────────────────────
# 3) Yearly engagements by influencer & side = count of politician RTs
grp = (
    df.groupby([INFLUENCER_COL, "year", "side"], dropna=False)
      .size()
      .rename("engagements")
      .reset_index()
)

# 4) Pivot to ruling/opposition columns; missing → 0
pivot = (
    grp.pivot_table(index=[INFLUENCER_COL, "year"],
                    columns="side", values="engagements",
                    aggfunc="sum", fill_value=0)
      .reset_index()
)

for col in ["ruling", "opposition"]:
    if col not in pivot.columns:
        pivot[col] = 0

# 5) Yearly polarity: (R - O) / (R + O); undefined when R+O=0
den = (pivot["ruling"] + pivot["opposition"]).astype(float)
pivot["polarity_yearly"] = (pivot["ruling"] - pivot["opposition"]) / den
pivot.loc[den == 0, "polarity_yearly"] = pd.NA

# 6) Average polarity across years per influencer
influencer_polarity = (
    pivot.groupby(INFLUENCER_COL, dropna=False)["polarity_yearly"]
         .mean()
         .rename("polarity_avg")
         .reset_index()
)

# 7) Single-threshold labeling at 0.50
THRESHOLD = 0.50

def label_from_polarity(x, thr=THRESHOLD):
    if pd.isna(x):    return "Unlabeled"
    if x >= thr:      return "Pro Ruling"
    if x <= -thr:     return "Pro OPP"
    return "Neutral"

influencer_polarity["label_0_5"] = influencer_polarity["polarity_avg"].apply(label_from_polarity)

# 8) Attach influencer polarity/labels back to every tweet
df = df.merge(influencer_polarity, on=INFLUENCER_COL, how="left")

# Choose primary tweet label (0.50 threshold)
df["tweet_label"] = df["label_0_5"]

# ──────────────────────────────────────────────────────────────────────────────
# 9) Save outputs
out_dir = Path("outputs_final"); out_dir.mkdir(exist_ok=True)
pivot.to_csv(out_dir / "yearly_engagements_and_polarity.csv", index=False)
influencer_polarity.to_csv(out_dir / "influencer_polarity_scores.csv", index=False)
df.to_csv(out_dir / "tweets_with_labels.csv", index=False)

print("[done] Wrote:")
print(" - outputs_final/yearly_engagements_and_polarity.csv")
print(" - outputs_final/influencer_polarity_scores.csv")
print(" - outputs_final/tweets_with_labels.csv")

# Quick sanity summary
print("\nLabel counts @ 0.50:")
print(df["tweet_label"].value_counts(dropna=False))


[done] Wrote:
 - outputs_final/yearly_engagements_and_polarity.csv
 - outputs_final/influencer_polarity_scores.csv
 - outputs_final/tweets_with_labels.csv

Label counts @ 0.50:
tweet_label
Pro Ruling    3024136
Pro OPP       2649354
Neutral        969025
NaN            473448
Name: count, dtype: int64


In [4]:
df = pd.read_csv("/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels.csv")
df.head()

,timestamp,tweet,retweet_author,original_author,retweet_lc,original_lc,retweet_party,year,side,polarity_avg,label_0_5,tweet_label
0,2020-06-18 11:59:57+00:00,PM ⁦@narendramodi⁩ to launch Garib Kalyan Rojg...,BJP4AnN,PMOIndia,bjp4ann,pmoindia,BJP,2020.0,ruling,0.967184,Pro Ruling,Pro Ruling
1,2020-05-20 18:15:42+00:00,A step towards economic resilience #aatmanirbh...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP,2020.0,ruling,0.960328,Pro Ruling,Pro Ruling
2,2020-05-20 18:14:57+00:00,.@MOFPI_GOI wholeheartedly welcomes the Cabine...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP,2020.0,ruling,0.960328,Pro Ruling,Pro Ruling
3,2020-06-02 15:58:00+00:00,राज्यपाल भगतसिंह कोश्यारी यांनी मुख्यमंत्री उद...,Nitinkalebjp,maha_governor,nitinkalebjp,maha_governor,BJP,2020.0,ruling,0.893685,Pro Ruling,Pro Ruling
4,2020-12-15 09:47:14+00:00,VIDEO : Pune | केंद्रीय मंत्री प्रकाश जावडेकरा...,VartakKuldeep,TV9Marathi,vartakkuldeep,tv9marathi,INC,2020.0,opposition,-0.443838,Neutral,Neutral


In [5]:
import pandas as pd
from pathlib import Path
import re

# ── 0) Config ─────────────────────────────────────────────────────────────────
CSV_PATH = "/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels.csv"

# ── 1) Load ───────────────────────────────────────────────────────────────────
df = pd.read_csv(CSV_PATH, low_memory=False)

# ── 2) English filter ─────────────────────────────────────────────────────────
# Prefer a language column if present; otherwise use a fast heuristic on text.
lang_candidates = [c for c in df.columns if c.lower() in {
    "lang", "tweet_lang", "language", "tweet_language", "iso_language_code"
}]

def looks_english(text: str) -> bool:
    """
    Fast heuristic: treat as English if ≥85% of alphabetic chars are ASCII letters.
    URLs, mentions, hashtags, digits are ignored for the ratio.
    """
    if not isinstance(text, str) or not text.strip():
        return False
    t = re.sub(r"https?://\S+|@\w+|#\w+|\d+", " ", text)
    letters = sum(ch.isalpha() for ch in t)
    ascii_letters = sum(ch.isascii() and ch.isalpha() for ch in t)
    return letters > 0 and (ascii_letters / letters) >= 0.85

if lang_candidates:
    lang_col = lang_candidates[0]
    df_en = df[df[lang_col].astype(str).str.lower().eq("en")].copy()
    used_detector = f"language column '{lang_col}'"
else:
    # Fallback without external deps (fast, good for large datasets)
    text_col = "tweet" if "tweet" in df.columns else df.columns[0]
    df_en = df[df[text_col].map(looks_english)].copy()
    used_detector = "ASCII-ratio heuristic on tweet text"

# ── 3) Ensure 'year' exists for grouping if only timestamp is present ─────────
if "year" not in df_en.columns and "timestamp" in df_en.columns:
    df_en["timestamp"] = pd.to_datetime(df_en["timestamp"], errors="coerce", utc=True)
    df_en["year"] = df_en["timestamp"].dt.year

# ── 4) Save filtered CSV ──────────────────────────────────────────────────────
out_path = Path(CSV_PATH).with_name(Path(CSV_PATH).stem + "_en_1.csv")
df_en.to_csv(out_path, index=False)

# ── 5) Stance breakdowns (same style as your snippet, but on English-only) ────
# Pick the stance column (prefers tweet_label)
label_col = next((c for c in ["tweet_label", "label_0_5", "label_0_75", "label_0_3"] if c in df_en.columns), None)
if label_col is None:
    raise ValueError("No stance label column found (expected one of: tweet_label, label_0_5, label_0_75, label_0_3).")

def norm_label(x: str):
    if not isinstance(x, str):
        return "other"
    s = x.strip().lower()
    if "pro ruling" in s or s == "ruling":
        return "pro_ruling"
    if "pro opp" in s or "pro opposition" in s or s == "opposition":
        return "pro_opp"
    return "other"  # neutral/unknown/other

df_en["_stance"] = df_en[label_col].apply(norm_label)

# Overall counts
overall = df_en["_stance"].value_counts().reindex(["pro_ruling", "pro_opp", "other"], fill_value=0)

# Print header + save summary
total_before = len(df)
total_after = len(df_en)
pct = (total_after / max(total_before, 1)) * 100

print(f"Filtered English-only tweets using {used_detector}.")
print(f"Saved → {out_path}")
print(f"Kept {total_after:,} of {total_before:,} rows ({pct:.1f}%).\n")

print(f"=== Stance counts (from {label_col}) on English-only subset ===")
print(overall.to_string())

# By year (optional)
if "year" in df_en.columns:
    by_year = (
        df_en.groupby("year")["_stance"]
        .value_counts()
        .unstack(fill_value=0)
        .reindex(columns=["pro_ruling", "pro_opp", "other"], fill_value=0)
    )
    print("\n=== Stance counts by year (English-only) ===")
    print(by_year.head(20).to_string())

# Author side counts (optional)
if "side" in df_en.columns:
    side_counts = (
        df_en["side"].astype(str).str.strip().str.lower()
        .value_counts()
        .reindex(["ruling", "opposition"], fill_value=0)
    )
    print("\n=== Author side counts (from 'side' column) on English-only subset ===")
    print(side_counts.to_string())

# (Optional) If you prefer a model-based detector for higher accuracy on code-mixed text:
#   !pip install langdetect
#   from langdetect import detect
#   df['_lang'] = df['tweet'].apply(lambda t: detect(t) if isinstance(t, str) and t.strip() else None)
#   df_en = df[df['_lang'].eq('en')].copy()


Filtered English-only tweets using ASCII-ratio heuristic on tweet text.
Saved → /Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels_en_1.csv
Kept 2,939,716 of 7,115,963 rows (41.3%).

=== Stance counts (from tweet_label) on English-only subset ===
_stance
pro_ruling    1450353
pro_opp        934640
other          554723

=== Stance counts by year (English-only) ===
_stance  pro_ruling  pro_opp   other
year                                
2020.0       295292   226402  107387
2021.0       588350   416479  184407
2022.0       408093   207819  220925
2023.0       158618    83940   41931

=== Author side counts (from 'side' column) on English-only subset ===
side
ruling        1566235
opposition    1038855


In [6]:
df = pd.read_csv("/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels_en_1.csv")
df.head()

,timestamp,tweet,retweet_author,original_author,retweet_lc,original_lc,retweet_party,year,side,polarity_avg,label_0_5,tweet_label
0,2020-06-18 11:59:57+00:00,PM ⁦@narendramodi⁩ to launch Garib Kalyan Rojg...,BJP4AnN,PMOIndia,bjp4ann,pmoindia,BJP,2020.0,ruling,0.967184,Pro Ruling,Pro Ruling
1,2020-05-20 18:15:42+00:00,A step towards economic resilience #aatmanirbh...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP,2020.0,ruling,0.960328,Pro Ruling,Pro Ruling
2,2020-05-20 18:14:57+00:00,.@MOFPI_GOI wholeheartedly welcomes the Cabine...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP,2020.0,ruling,0.960328,Pro Ruling,Pro Ruling
3,2020-11-27 08:43:45+00:00,Russia agrees to produce coronavirus vaccine S...,VartakKuldeep,timesofindia,vartakkuldeep,timesofindia,INC,2020.0,opposition,-0.120152,Neutral,Neutral
4,2020-11-28 05:31:01+00:00,Overwhelmed and deeply moved by everyone’s con...,VartakKuldeep,mfaisalpatel,vartakkuldeep,mfaisalpatel,INC,2020.0,opposition,-0.985450,Pro OPP,Pro OPP


In [7]:
import pandas as pd
import re
import hashlib
from pathlib import Path

# ── 0) Paths ──────────────────────────────────────────────────────────────────
OLD_PATH = Path("/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/2_Keyword_extractor/visualisations/tweets_with_subjects.csv")
NEW_PATH = Path("/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels_en_1.csv")
OUT_PATH = NEW_PATH.with_name(NEW_PATH.stem + "_with_subjects.csv")

# ── 1) Load ───────────────────────────────────────────────────────────────────
old_df = pd.read_csv(OLD_PATH, low_memory=False)
new_df = pd.read_csv(NEW_PATH, low_memory=False)

# ── 2) Pick tweet column names ────────────────────────────────────────────────
def get_tweet_col(df):
    for c in df.columns:
        if c.lower() == "tweet":
            return c
    for alt in ("text", "content"):
        if alt in df.columns:
            return alt
    return df.columns[0]

tweet_col_old = get_tweet_col(old_df)
tweet_col_new = get_tweet_col(new_df)

# ── 3) Normalization + stable key ─────────────────────────────────────────────
_URL_RE = re.compile(r"https?://\S+")
_AT_HASH_RE = re.compile(r"[@#]\w+")
_HTML_ENT_RE = re.compile(r"&(?:amp|lt|gt);")
_NON_WORD_KEEP_SPACE = re.compile(r"[^\w\s]")   # keep letters/digits/_/space
_MULTISPACE_RE = re.compile(r"\s+")

def normalize_tweet(text: str) -> str:
    t = str(text).lower()
    t = _URL_RE.sub(" ", t)
    t = _AT_HASH_RE.sub(" ", t)
    t = _HTML_ENT_RE.sub(" ", t)
    t = _NON_WORD_KEEP_SPACE.sub(" ", t)
    t = _MULTISPACE_RE.sub(" ", t).strip()
    return t

def make_key(text: str) -> str:
    return hashlib.sha1(normalize_tweet(text).encode("utf-8")).hexdigest()

old_df["__key__"] = old_df[tweet_col_old].map(make_key)
new_df["__key__"] = new_df[tweet_col_new].map(make_key)

# ── 4) Build lookup of subjects from OLD ──────────────────────────────────────
need_cols = ["subjects", "subjects_scored"]
missing = [c for c in need_cols if c not in old_df.columns]
if missing:
    raise KeyError(f"Missing in OLD CSV: {missing}")

# If duplicates exist, prefer the first non-null subjects row per key
old_subj = (
    old_df
    .sort_values(by=["__key__"])  # stable
    .drop_duplicates(subset="__key__", keep="first")
    .set_index("__key__")[need_cols]
)

# ── 5) Merge onto NEW (left join) ─────────────────────────────────────────────
merged = new_df.merge(old_subj, how="left", left_on="__key__", right_index=True)

# ── 6) Report & save ─────────────────────────────────────────────────────────
matched = merged["subjects"].notna().sum()
total_new = len(merged)
print("=== Append subjects → NEW ===")
print(f"NEW rows total        : {total_new:,}")
print(f"Rows matched (subjects): {matched:,} ({matched/total_new:.2%})")

# Clean up helper key
merged = merged.drop(columns=["__key__"])

merged.to_csv(OUT_PATH, index=False)
print(f"Saved → {OUT_PATH}")


=== Append subjects → NEW ===
NEW rows total        : 2,939,716
Rows matched (subjects): 2,939,716 (100.00%)
Saved → /Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels_en_1_with_subjects.csv


In [8]:
df = pd.read_csv("/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels_en_1_with_subjects.csv")
df.head()

,timestamp,tweet,retweet_author,original_author,retweet_lc,original_lc,retweet_party,year,side,polarity_avg,label_0_5,tweet_label,subjects,subjects_scored
0,2020-06-18 11:59:57+00:00,PM ⁦@narendramodi⁩ to launch Garib Kalyan Rojg...,BJP4AnN,PMOIndia,bjp4ann,pmoindia,BJP,2020.0,ruling,0.967184,Pro Ruling,Pro Ruling,"['narendramodi', 'launch', 'livelihood']","[{'text': 'narendramodi', 'score': 0.4989}, {'..."
1,2020-05-20 18:15:42+00:00,A step towards economic resilience #aatmanirbh...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP,2020.0,ruling,0.960328,Pro Ruling,Pro Ruling,"['resilience', '25000', 'food']","[{'text': 'resilience', 'score': 0.517}, {'tex..."
2,2020-05-20 18:14:57+00:00,.@MOFPI_GOI wholeheartedly welcomes the Cabine...,BJP4AnN,MOFPI_GOI,bjp4ann,mofpi_goi,BJP,2020.0,ruling,0.960328,Pro Ruling,Pro Ruling,"['mofpi_goi', 'cabinet', 'food']","[{'text': 'mofpi_goi', 'score': 0.4442}, {'tex..."
3,2020-11-27 08:43:45+00:00,Russia agrees to produce coronavirus vaccine S...,VartakKuldeep,timesofindia,vartakkuldeep,timesofindia,INC,2020.0,opposition,-0.120152,Neutral,Neutral,"['coronavirus', 'russia', 'india']","[{'text': 'coronavirus', 'score': 0.4371}, {'t..."
4,2020-11-28 05:31:01+00:00,Overwhelmed and deeply moved by everyone’s con...,VartakKuldeep,mfaisalpatel,vartakkuldeep,mfaisalpatel,INC,2020.0,opposition,-0.985450,Pro OPP,Pro OPP,"['condolence', 'whatsapp', 'immediately']","[{'text': 'condolence', 'score': 0.5105}, {'te..."


In [9]:
#tweet tweet_label subjects subjects_scored
import pandas as pd
from pathlib import Path

SRC = Path("/Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/tweets_with_labels_en_1_with_subjects.csv")
OUT = SRC.with_name("final_before_stance.csv")

keep_cols = ["tweet", "tweet_label", "subjects", "subjects_scored"]

df = pd.read_csv(SRC, low_memory=False)

# Ensure columns exist (and keep correct order)
missing = [c for c in keep_cols if c not in df.columns]
if missing:
    raise KeyError(f"Missing expected columns in source CSV: {missing}")

df[keep_cols].to_csv(OUT, index=False)
print(f"Saved → {OUT}")


Saved → /Users/ziv/Desktop/PARTISAN_FINAL_PIPELINE/1_DATA_FORMATTING/scripts/outputs_final/final_before_stance.csv
